In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pickle

In [3]:
#중고나라 크롤링한 Pickle파일 가져오기
df = pickle.load(open("../LgNotebook_joongonara.pkl","rb"))

In [4]:
#노트북 모델명 찾는 함수
import re
def find_model_name(content):
    pattern = re.compile("[a-zA-Z0-9]{4,8}[-][a-zA-Z0-9]{5,6}")
    model_names = pattern.findall(content)
    if model_names:
        return model_names[0]
    return "None"

# 노트북 중고가격 찾는 함수
import re
def find_price(content):
    pattern = re.compile("[0-9, ]{1,10}[ 만원마넌]{1,3}")
    old_price = pattern.findall(content)
    if old_price:
        return old_price[0]
    return "None"

# 노트북 출시 날짜 가져오기
import re
def find_aftrer_released_month(content):
    pattern = re.compile("[0-9]{4}[.][0-9]{1,2}")
    released_month = pattern.findall(content)
    if released_month:
        return released_month[0]
    return "None"


In [5]:
# 다나와 페이지에서 모델명 검색해서 스펙가져오는 함수
def get_info(code):
    try:
        url = "http://search.danawa.com/ajax/getProductList.ajax.php"
        headers = {
            "Accept": "text/html, */*; q=0.01",
            "Accept-Encoding": "gzip, deflate",
            "Accept-Language": "ko-KR",
            "Connection": "Keep-Alive",
            "Content-Length": "145",
            "Content-Type": "application/x-www-form-urlencoded",
            "Cookie":"OAX=AeNIfFd3bFYAA6++; danawa-loggingApplicationClient=33443301-8e37-46dc-8efa-59b9175e20cc; ACEFCID=UID-577773FCE7F5EEAA51E1EB8F; ADWEBCOUNTER_UUID=cbf28ded-957a-20ea-a873-2ad7c1cb0238; cookSaveShopInfo=EE128%3A2016-07-02%7CPO814%3A2016-07-08; _ga=GA1.2.1089033139.1467444311; cookieGuestId=6e0e50e54ff9b2562ab7f7cd2b418748; userAccessIP=59.29.231.9; cPreviousKeyword=15ZD960-GX50K; userAccessTime=2016-08-01+18%3A17%3A45; _EXEN=2; _INSIGHT_CK_8203=af985b20b108d0247168c9e1a98961a1_43057|5121f5117fbe615e6215c9e1a98961a1_43057:1470044866000; ACEUCI=1; wcs_bt=s_3b3fb74948b1:1470043066; ADWEBCOUNTER_KEYWORD=; ADWEBCOUNTER_URL=; cookSearchKeyword=15ZD960-GX50K",
            "Host": "search.danawa.com",
            "Referer": "http://search.danawa.com/dsearch.php?k1=15ZD960-GX50K&module=goods&act=dispMain",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36",
            "X-Requested-With": "XMLHttpRequest",
        }
        data = {
            "query":code,
            "originalQuery":code,
            "tab":"main",
            "volumeType":"allvs",
            "page":"1",
            "limit":"0",
            "sort":"",
            "list":"list",
            "boost":"true"
        }

        response = requests.post(url, data=data, headers=headers)
        dom = BeautifulSoup(response.content, "html.parser")
        data = [dom.select_one("div.prod_sub_meta dd").text , dom.select_one("div.spec_list").text]
        return data
    except:
        return [code, "none"]


In [6]:
df["Model_name"] = df["Content"].astype("str").apply(find_model_name)
df1 = df[df["Model_name"] != "None"]

In [167]:
df1["Model_spec"] = df["Model_name"].apply(get_info)

C:\Users\suk\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [168]:
df1.head()

,Title,URL,Content,Model_name,Model_spec
1,"[경기 파주] LG X-NOTE S525-KE50K(램 8GB, SSD 250GB ...",http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,S525-KE50K,"[2012.01, 인텔 / 코어i5-2세대 / 샌디브릿지 / i5-2450M (2...."
2,"[부산,경남,창원 직거래]LG노트북 15U550-KT5MK 판매합니다.",http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,15U550-KT5MK,"[2016.01, 인텔 / 코어i5-5세대 / 브로드웰 / i5-5200U (2.2..."
4,LG PC 그램 노트북 14Z950-GT50K 판매합니다,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,14Z950-GT50K,"[2015.01, 인텔 / 코어i5-5세대 / 브로드웰 / i5-5200U (2.2..."
7,LG전자 15UD560-KX7DK[+SSD 256G] 노트북 판매합니다,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,15UD560-KX7DK,"[2015.12, 인텔 / 코어i7-6세대 / 스카이레이크 / i7-6500U (2..."
10,LG 15U340-LT1FK 화이트 노트북 팔아요 가격 내림,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,LG15U340-LT1FK,"[2014.06, 인텔 / 펜티엄 / 베이트레일 / N3520 (2.166GHz) ..."


In [169]:
df1["Released_date"] = df1["Model_spec"].astype("str").apply(find_aftrer_released_month)

C:\Users\suk\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [174]:
df1.head(10)

,Title,URL,Content,Model_name,Model_spec,Released_date
1,"[경기 파주] LG X-NOTE S525-KE50K(램 8GB, SSD 250GB ...",http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,S525-KE50K,"[2012.01, 인텔 / 코어i5-2세대 / 샌디브릿지 / i5-2450M (2....",2012.01
2,"[부산,경남,창원 직거래]LG노트북 15U550-KT5MK 판매합니다.",http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,15U550-KT5MK,"[2016.01, 인텔 / 코어i5-5세대 / 브로드웰 / i5-5200U (2.2...",2016.01
4,LG PC 그램 노트북 14Z950-GT50K 판매합니다,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,14Z950-GT50K,"[2015.01, 인텔 / 코어i5-5세대 / 브로드웰 / i5-5200U (2.2...",2015.01
7,LG전자 15UD560-KX7DK[+SSD 256G] 노트북 판매합니다,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,15UD560-KX7DK,"[2015.12, 인텔 / 코어i7-6세대 / 스카이레이크 / i7-6500U (2...",2015.12
10,LG 15U340-LT1FK 화이트 노트북 팔아요 가격 내림,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,LG15U340-LT1FK,"[2014.06, 인텔 / 펜티엄 / 베이트레일 / N3520 (2.166GHz) ...",2014.06
12,판매완료-[서울30만원판매] LG전자 엑스노트 P210-GE30K / 윈도우7 초기...,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,P210-GE25K,"[2015.03, 보호용품 / 키스킨]",2015.03
16,LG노트북 팔아요 (15ND540-UX50K),http://cafe.naver.com/ArticleRead.nhn?clubid=1...,LG노트북 팔아요 15ND540-UX50K\n현재 99만원에 최저가 올라와있더라구요...,15ND540-UX50K,"[2014.08, 인텔 / 코어i5-4세대 / 하스웰 / i5-4210M (2.6G...",2014.08
18,"삼성,LG노트북팔아요^^",http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,z160-gh30k,"[2013.02, 인텔 / 코어i3-3세대 / 아이비브릿지 / i3-3217U (1...",2013.02
22,[대구] LG노트북 15UD560-KX50K 팜니다,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,15UD560-KX50K,"[2015.12, 인텔 / 코어i5-6세대 / 스카이레이크 / i5-6200U (2...",2015.12
31,부산 lg 노트북 그램 13 (13Z950-LR10K)(60만원),http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,13Z950-LR10K,"[2016.01, 인텔 / 펜티엄 / 브로드웰 / 3805U (1.9GHz) / 3...",2016.01


In [177]:
import datetime
def find_how_months(content):
    released_date = datetime.date(int(content.split(".")[0]),int(content.split(".")[1]),1)
    base_date = datetime.date(2016,7,1)
    
    return round(((base_date - released_date).days)/30)


In [181]:
df1 = df1[df1["Released_date"] != "None"]

In [182]:
df1["After_released_date"] = df1["Released_date"].apply(find_how_months)

C:\Users\suk\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [183]:
df1.head(10)

,Title,URL,Content,Model_name,Model_spec,Released_date,After_released_date
1,"[경기 파주] LG X-NOTE S525-KE50K(램 8GB, SSD 250GB ...",http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,S525-KE50K,"[2012.01, 인텔 / 코어i5-2세대 / 샌디브릿지 / i5-2450M (2....",2012.01,55
2,"[부산,경남,창원 직거래]LG노트북 15U550-KT5MK 판매합니다.",http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,15U550-KT5MK,"[2016.01, 인텔 / 코어i5-5세대 / 브로드웰 / i5-5200U (2.2...",2016.01,6
4,LG PC 그램 노트북 14Z950-GT50K 판매합니다,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,14Z950-GT50K,"[2015.01, 인텔 / 코어i5-5세대 / 브로드웰 / i5-5200U (2.2...",2015.01,18
7,LG전자 15UD560-KX7DK[+SSD 256G] 노트북 판매합니다,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,15UD560-KX7DK,"[2015.12, 인텔 / 코어i7-6세대 / 스카이레이크 / i7-6500U (2...",2015.12,7
10,LG 15U340-LT1FK 화이트 노트북 팔아요 가격 내림,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,LG15U340-LT1FK,"[2014.06, 인텔 / 펜티엄 / 베이트레일 / N3520 (2.166GHz) ...",2014.06,25
12,판매완료-[서울30만원판매] LG전자 엑스노트 P210-GE30K / 윈도우7 초기...,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,P210-GE25K,"[2015.03, 보호용품 / 키스킨]",2015.03,16
16,LG노트북 팔아요 (15ND540-UX50K),http://cafe.naver.com/ArticleRead.nhn?clubid=1...,LG노트북 팔아요 15ND540-UX50K\n현재 99만원에 최저가 올라와있더라구요...,15ND540-UX50K,"[2014.08, 인텔 / 코어i5-4세대 / 하스웰 / i5-4210M (2.6G...",2014.08,23
18,"삼성,LG노트북팔아요^^",http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,z160-gh30k,"[2013.02, 인텔 / 코어i3-3세대 / 아이비브릿지 / i3-3217U (1...",2013.02,42
22,[대구] LG노트북 15UD560-KX50K 팜니다,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,15UD560-KX50K,"[2015.12, 인텔 / 코어i5-6세대 / 스카이레이크 / i5-6200U (2...",2015.12,7
31,부산 lg 노트북 그램 13 (13Z950-LR10K)(60만원),http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,13Z950-LR10K,"[2016.01, 인텔 / 펜티엄 / 브로드웰 / 3805U (1.9GHz) / 3...",2016.01,6


In [184]:
df1["Old_price"] = df1["Content"].apply(find_price)

C:\Users\suk\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [199]:
#index 정리
df2 = df1.reset_index(drop=True)

In [186]:
df2.head()

,Title,URL,Content,Model_name,Model_spec,Released_date,After_released_date,Old_price
0,"[경기 파주] LG X-NOTE S525-KE50K(램 8GB, SSD 250GB ...",http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,S525-KE50K,"[2012.01, 인텔 / 코어i5-2세대 / 샌디브릿지 / i5-2450M (2....",2012.01,55,"380,000 원"
1,"[부산,경남,창원 직거래]LG노트북 15U550-KT5MK 판매합니다.",http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,15U550-KT5MK,"[2016.01, 인텔 / 코어i5-5세대 / 브로드웰 / i5-5200U (2.2...",2016.01,6,"700,000 원"
2,LG PC 그램 노트북 14Z950-GT50K 판매합니다,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,14Z950-GT50K,"[2015.01, 인텔 / 코어i5-5세대 / 브로드웰 / i5-5200U (2.2...",2015.01,18,1 원
3,LG전자 15UD560-KX7DK[+SSD 256G] 노트북 판매합니다,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,15UD560-KX7DK,"[2015.12, 인텔 / 코어i7-6세대 / 스카이레이크 / i7-6500U (2...",2015.12,7,120 원
4,LG 15U340-LT1FK 화이트 노트북 팔아요 가격 내림,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,LG15U340-LT1FK,"[2014.06, 인텔 / 펜티엄 / 베이트레일 / N3520 (2.166GHz) ...",2014.06,25,"350,000 원"


In [194]:
df3 = df2[df2["Old_price"].str.len()>4]

In [195]:
df3

,Title,URL,Content,Model_name,Model_spec,Released_date,After_released_date,Old_price
0,"[경기 파주] LG X-NOTE S525-KE50K(램 8GB, SSD 250GB ...",http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,S525-KE50K,"[2012.01, 인텔 / 코어i5-2세대 / 샌디브릿지 / i5-2450M (2....",2012.01,55,"380,000 원"
1,"[부산,경남,창원 직거래]LG노트북 15U550-KT5MK 판매합니다.",http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,15U550-KT5MK,"[2016.01, 인텔 / 코어i5-5세대 / 브로드웰 / i5-5200U (2.2...",2016.01,6,"700,000 원"
3,LG전자 15UD560-KX7DK[+SSD 256G] 노트북 판매합니다,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,15UD560-KX7DK,"[2015.12, 인텔 / 코어i7-6세대 / 스카이레이크 / i7-6500U (2...",2015.12,7,120 원
4,LG 15U340-LT1FK 화이트 노트북 팔아요 가격 내림,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,LG15U340-LT1FK,"[2014.06, 인텔 / 펜티엄 / 베이트레일 / N3520 (2.166GHz) ...",2014.06,25,"350,000 원"
5,판매완료-[서울30만원판매] LG전자 엑스노트 P210-GE30K / 윈도우7 초기...,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,P210-GE25K,"[2015.03, 보호용품 / 키스킨]",2015.03,16,28 원
6,LG노트북 팔아요 (15ND540-UX50K),http://cafe.naver.com/ArticleRead.nhn?clubid=1...,LG노트북 팔아요 15ND540-UX50K\n현재 99만원에 최저가 올라와있더라구요...,15ND540-UX50K,"[2014.08, 인텔 / 코어i5-4세대 / 하스웰 / i5-4210M (2.6G...",2014.08,23,99만원
7,"삼성,LG노트북팔아요^^",http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,z160-gh30k,"[2013.02, 인텔 / 코어i3-3세대 / 아이비브릿지 / i3-3217U (1...",2013.02,42,10 원
8,[대구] LG노트북 15UD560-KX50K 팜니다,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,15UD560-KX50K,"[2015.12, 인텔 / 코어i5-6세대 / 스카이레이크 / i5-6200U (2...",2015.12,7,"800,000 원"
9,부산 lg 노트북 그램 13 (13Z950-LR10K)(60만원),http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,13Z950-LR10K,"[2016.01, 인텔 / 펜티엄 / 브로드웰 / 3805U (1.9GHz) / 3...",2016.01,6,"600,000 원"
10,"LG 15인치 노트북 15만원(듀얼P8600, 4G, 250GB, led)",http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,s510-up86k,"[2008.07, 인텔 / 코어2듀오(펜린) / 39.11cm(15.4인치) / 1...",2008.07,97,"160,000 원"


In [196]:
df3 = df3.drop([12,290,318])

In [197]:
df3["Old_price"] = df3["Old_price"].apply(lambda x: int(x[:-1].replace(",","")) if len(x)>6 \
                                          else (  int(x[:-1])*10000 if x.find("만")<0 else int(x[:-2])*10000 ))

In [200]:
df3.drop([302])

,Title,URL,Content,Model_name,Model_spec,Released_date,After_released_date,Old_price
0,"[경기 파주] LG X-NOTE S525-KE50K(램 8GB, SSD 250GB ...",http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,S525-KE50K,"[2012.01, 인텔 / 코어i5-2세대 / 샌디브릿지 / i5-2450M (2....",2012.01,55,380000
1,"[부산,경남,창원 직거래]LG노트북 15U550-KT5MK 판매합니다.",http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,15U550-KT5MK,"[2016.01, 인텔 / 코어i5-5세대 / 브로드웰 / i5-5200U (2.2...",2016.01,6,700000
3,LG전자 15UD560-KX7DK[+SSD 256G] 노트북 판매합니다,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,15UD560-KX7DK,"[2015.12, 인텔 / 코어i7-6세대 / 스카이레이크 / i7-6500U (2...",2015.12,7,1200000
4,LG 15U340-LT1FK 화이트 노트북 팔아요 가격 내림,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,LG15U340-LT1FK,"[2014.06, 인텔 / 펜티엄 / 베이트레일 / N3520 (2.166GHz) ...",2014.06,25,350000
5,판매완료-[서울30만원판매] LG전자 엑스노트 P210-GE30K / 윈도우7 초기...,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,P210-GE25K,"[2015.03, 보호용품 / 키스킨]",2015.03,16,280000
6,LG노트북 팔아요 (15ND540-UX50K),http://cafe.naver.com/ArticleRead.nhn?clubid=1...,LG노트북 팔아요 15ND540-UX50K\n현재 99만원에 최저가 올라와있더라구요...,15ND540-UX50K,"[2014.08, 인텔 / 코어i5-4세대 / 하스웰 / i5-4210M (2.6G...",2014.08,23,990000
7,"삼성,LG노트북팔아요^^",http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,z160-gh30k,"[2013.02, 인텔 / 코어i3-3세대 / 아이비브릿지 / i3-3217U (1...",2013.02,42,100000
8,[대구] LG노트북 15UD560-KX50K 팜니다,http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,15UD560-KX50K,"[2015.12, 인텔 / 코어i5-6세대 / 스카이레이크 / i5-6200U (2...",2015.12,7,800000
9,부산 lg 노트북 그램 13 (13Z950-LR10K)(60만원),http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,13Z950-LR10K,"[2016.01, 인텔 / 펜티엄 / 브로드웰 / 3805U (1.9GHz) / 3...",2016.01,6,600000
10,"LG 15인치 노트북 15만원(듀얼P8600, 4G, 250GB, led)",http://cafe.naver.com/ArticleRead.nhn?clubid=1...,안전거래 미사용 게시물입니다.\n본 게시물은 에스크로를 이용하지 않으므로 거래 안전...,s510-up86k,"[2008.07, 인텔 / 코어2듀오(펜린) / 39.11cm(15.4인치) / 1...",2008.07,97,160000


In [201]:
#df3.to_pickle("lg_JoongoNotebook.pkl")